## Loading Embeddings

The models are pickled as a python tuple.

The first element of the pair is a python list. The list represents the vocabulary words sorted by their frequency in the corpus.

The second element is a python numpy array where each row represents a word vector. The embeddings are stored as 32 bit float numpy array to save space.

In [3]:
import pickle
import numpy
words, embeddings = pickle.load(open('polyglot-pl.pkl', 'rb'))
print("Emebddings shape is {}".format(embeddings.shape))

Emebddings shape is (238004, 64)


The size of the emebddings tells about the number of words that make up the vocabulary (100004) and the size of the vector that represent each word (64).

The vocabulary consist of 100004 words. The first four words are special symbols:

In [4]:
Token_ID = {"<UNK>": 0, "<S>": 1, "</S>":2, "<PAD>": 3}
#{<UNK>: Out of vocabulary word,
# <S>: Start of sentence,
# </S>: End of sentence,
# <PAD>: Padding character}.

If your sentence is "I visited New York .", then the model was trained on the following five 5-grams:

In [5]:
(("<PAD>", "<S>", "I", "visited", "New"),
 ("<S>", "I", "visited", "New", "York"),
 ("I", "visited", "New", "York", "."),
 ("visited", "New", "York", "." "</S>"),
 ("New", "York", ".", "</S>", "<PAD>"))

(('<PAD>', '<S>', 'I', 'visited', 'New'),
 ('<S>', 'I', 'visited', 'New', 'York'),
 ('I', 'visited', 'New', 'York', '.'),
 ('visited', 'New', 'York', '.</S>'),
 ('New', 'York', '.', '</S>', '<PAD>'))

In [6]:
# First ten words in the vocabulary. Notice the special symbols <UNK> ... <PAD>
words[:10]

(u'</S>', u'<UNK>', u'<PAD>', u'<S>', u'.', u',', u'w', u'####', u'i', u'|')

In [7]:
# A tuple of the word "outside" and its representation as a python array
words[777], embeddings[777]

(u'XVII',
 array([ 0.30604595,  0.41035393,  0.03496124,  0.01761231, -0.10205037,
        -0.0510378 , -0.27494964, -0.33645159, -0.35410404,  0.20965298,
        -0.01945674, -0.35793582,  0.08840241, -0.22543047, -0.16240528,
        -0.03040629,  0.09480856,  0.07561062, -0.33731684,  0.17146568,
        -0.1934309 , -0.03382664,  0.03248892,  0.26743773, -0.32673347,
         0.30685291,  0.40575019,  0.03751688,  0.23440546,  0.18500492,
        -0.26613352,  0.41453618, -0.01022074,  0.88271016,  0.14520411,
        -0.07502948, -0.58701319,  0.28594956, -0.10707291,  0.48185703,
         0.43553448, -0.1913278 ,  0.07920653,  0.63670272, -0.11858771,
         0.43688083, -0.14816599, -0.01360146, -0.65130663,  0.10476141,
        -0.25762817, -0.85841662, -0.19906059, -0.47304478,  0.18472129,
         0.45671263, -0.43421522,  0.16277687,  0.22997072, -0.04736441,
        -0.29756683, -0.45438197,  0.00403967, -0.37663564], dtype=float32))

# K-Nearest Neighbors Example

In [17]:
"""KNN Example."""

from operator import itemgetter
from itertools import izip, islice
import re
import numpy

# Special tokens
Token_ID = {"<UNK>": 0, "<S>": 1, "</S>":2, "<PAD>": 3}
ID_Token = {v:k for k,v in Token_ID.iteritems()}

# Map words to indices and vice versa
word_id = {w:i for (i, w) in enumerate(words)}
id_word = dict(enumerate(words))

# Noramlize digits by replacing them with #
DIGITS = re.compile("[0-9]", re.UNICODE)

# Number of neighbors to return.
k = 5


def case_normalizer(word, dictionary):
  """ In case the word is not available in the vocabulary,
     we can try multiple case normalizing procedure.
     We consider the best substitute to be the one with the lowest index,
     which is equivalent to the most frequent alternative."""
  w = word
  lower = (dictionary.get(w.lower(), 1e12), w.lower())
  upper = (dictionary.get(w.upper(), 1e12), w.upper())
  title = (dictionary.get(w.title(), 1e12), w.title())
  results = [lower, upper, title]
  results.sort()
  index, w = results[0]
  if index != 1e12:
    return w
  return word


def normalize(word, word_id):
    """ Find the closest alternative in case the word is OOV."""
    if not word in word_id:
        word = DIGITS.sub("#", word)
    if not word in word_id:
        word = case_normalizer(word, word_id)

    if not word in word_id:
        return None
    return word


def l2_nearest(embeddings, word_index, k):
    """Sorts words according to their Euclidean distance.
       To use cosine distance, embeddings has to be normalized so that their l2 norm is 1."""

    e = embeddings[word_index]
    distances = (((embeddings - e) ** 2).sum(axis=1) ** 0.5)
    sorted_distances = sorted(enumerate(distances), key=itemgetter(1))
    return zip(*sorted_distances[:k])


def knn(word, embeddings, word_id, id_word):
    word = normalize(word, word_id)
    if not word:
        print("OOV word")
        return
    word_index = word_id[word]
    indices, distances = l2_nearest(embeddings, word_index, k)
    neighbors = [id_word[idx] for idx in indices]
    for i, (word, distance) in enumerate(izip(neighbors, distances)):
      print i, '\t', word, '\t\t', distance

knn("putin", embeddings, word_id, id_word)
print

0 	Putin 		0.0
1 	Łukaszenka 		0.937288
2 	Rokossowski 		0.977874
3 	Daszyński 		0.982342
4 	Janukowycz 		0.984169



In [12]:
knn("zielony", embeddings, word_id, id_word)

0 	zielony 		0.0
1 	niebieski 		0.768414
2 	czarny 		0.976319
3 	błękitny 		1.03376
4 	żółty 		1.03414


In [16]:
knn("", embeddings, word_id, id_word)

OOV word
